In [1]:
import pandas as pd

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

import pickle

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [4]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi")

<Experiment: artifact_location='./mlruns/1', creation_time=1675147443326, experiment_id='1', last_update_time=1675147443326, lifecycle_stage='active', name='nyc-taxi', tags={}>

In [5]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df.lpep_pickup_datetime=pd.to_datetime(df.lpep_pickup_datetime)
    df.lpep_dropoff_datetime=pd.to_datetime(df.lpep_dropoff_datetime)
    # Create duration column
    df['duration']=df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60 )

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID","DOLocationID"]
    numerical = ["trip_distance"]

    df[categorical]=df[categorical].astype(str)
    return df

In [6]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [7]:
len(df_train), len(df_val)

(73908, 61921)

In [8]:
df_train['PU_DO'] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val['PU_DO'] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [9]:
df_val

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration,PU_DO
0,2,2021-02-01 00:34:03,2021-02-01 00:51:58,N,1.0,130,205,5.0,3.66,14.00,...,10.00,0.0,None,0.3,25.30,1.0,1.0,0.00,17.916667,130_205
1,2,2021-02-01 00:04:00,2021-02-01 00:10:30,N,1.0,152,244,1.0,1.10,6.50,...,0.00,0.0,None,0.3,7.80,2.0,1.0,0.00,6.500000,152_244
2,2,2021-02-01 00:18:51,2021-02-01 00:34:06,N,1.0,152,48,1.0,4.93,16.50,...,0.00,0.0,None,0.3,20.55,2.0,1.0,2.75,15.250000,152_48
3,2,2021-02-01 00:53:27,2021-02-01 01:11:41,N,1.0,152,241,1.0,6.70,21.00,...,0.00,0.0,None,0.3,22.30,2.0,1.0,0.00,18.233333,152_241
4,2,2021-02-01 00:57:46,2021-02-01 01:06:44,N,1.0,75,42,1.0,1.89,8.50,...,2.45,0.0,None,0.3,12.25,1.0,1.0,0.00,8.966667,75_42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64567,2,2021-02-28 22:19:00,2021-02-28 22:29:00,None,NaN,129,7,NaN,2.63,10.04,...,0.00,0.0,None,0.3,10.34,NaN,NaN,NaN,10.000000,129_7
64568,2,2021-02-28 23:18:00,2021-02-28 23:27:00,None,NaN,116,166,NaN,1.87,8.33,...,1.89,0.0,None,0.3,10.52,NaN,NaN,NaN,9.000000,116_166
64569,2,2021-02-28 23:44:00,2021-02-28 23:58:00,None,NaN,74,151,NaN,2.40,12.61,...,0.00,0.0,None,0.3,12.91,NaN,NaN,NaN,14.000000,74_151
64570,2,2021-02-28 23:07:00,2021-02-28 23:14:00,None,NaN,42,42,NaN,1.11,11.95,...,0.00,0.0,None,0.3,15.00,NaN,NaN,NaN,7.000000,42_42


In [10]:
categorical = ['PU_DO'] #"PULocationID","DOLocationID"
numerical = ["trip_distance"]
    
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [11]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [12]:

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715205895199

In [13]:
with open('models/lin_reg.bin','wb') as f_out:
    pickle.dump((dv,lr), f_out)

In [22]:
with mlflow.start_run():
    
    mlflow.set_tag("developer","Drax")
    
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
    
    alpha=0.01
    mlflow.log_param("alpha",alpha)
    
    lasso = Lasso(alpha=0.1)
    lasso.fit(X_train, y_train)

    y_pred = lasso.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse",rmse)

In [23]:

ridge = Ridge()
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.342603943231458

In [14]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope



In [15]:


train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)



In [16]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [27]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[05:37:22] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:17.41407                                                    
[1]	validation-rmse:14.53640                                                    
[2]	validation-rmse:12.37020                                                    
[3]	validation-rmse:10.76419                                                    
[4]	validation-rmse:9.58738                                                     
[5]	validation-rmse:8.74396                                                     
[6]	validation-rmse:8.13720                                                     
[7]	validation-rmse:7.70559                                                     
[8]	validation-rmse:7.39255                                                     
[9]	validation-rmse:7.17425                                                     
[10]	validation-rmse:7.01754                                           

[200]	validation-rmse:6.43108                                                   
[201]	validation-rmse:6.43036                                                   
[202]	validation-rmse:6.43037                                                   
[203]	validation-rmse:6.43052                                                   
[204]	validation-rmse:6.43055                                                   
[205]	validation-rmse:6.43012                                                   
[206]	validation-rmse:6.43025                                                   
[207]	validation-rmse:6.43119                                                   
[208]	validation-rmse:6.43065                                                   
[209]	validation-rmse:6.43044                                                   
[210]	validation-rmse:6.43040                                                   
[211]	validation-rmse:6.42995                                                   
[212]	validation-rmse:6.4294

[402]	validation-rmse:6.39937                                                   
[403]	validation-rmse:6.39928                                                   
[404]	validation-rmse:6.39899                                                   
[405]	validation-rmse:6.39928                                                   
[406]	validation-rmse:6.39953                                                   
[407]	validation-rmse:6.39948                                                   
[408]	validation-rmse:6.39947                                                   
[409]	validation-rmse:6.39954                                                   
[410]	validation-rmse:6.40017                                                   
[411]	validation-rmse:6.40021                                                   
[412]	validation-rmse:6.40036                                                   
[413]	validation-rmse:6.40053                                                   
[414]	validation-rmse:6.4003

[147]	validation-rmse:6.66976                                                   
[148]	validation-rmse:6.66960                                                   
[149]	validation-rmse:6.66948                                                   
[150]	validation-rmse:6.66931                                                   
[151]	validation-rmse:6.66921                                                   
[152]	validation-rmse:6.66912                                                   
[153]	validation-rmse:6.66898                                                   
[154]	validation-rmse:6.66881                                                   
[155]	validation-rmse:6.66874                                                   
[156]	validation-rmse:6.66866                                                   
[157]	validation-rmse:6.66842                                                   
[158]	validation-rmse:6.66834                                                   
[159]	validation-rmse:6.6681

[349]	validation-rmse:6.65006                                                   
[350]	validation-rmse:6.64995                                                   
[351]	validation-rmse:6.64989                                                   
[352]	validation-rmse:6.64981                                                   
[353]	validation-rmse:6.64972                                                   
[354]	validation-rmse:6.64972                                                   
[355]	validation-rmse:6.64966                                                   
[356]	validation-rmse:6.64959                                                   
[357]	validation-rmse:6.64957                                                   
[358]	validation-rmse:6.64947                                                   
[359]	validation-rmse:6.64946                                                   
[360]	validation-rmse:6.64937                                                   
[361]	validation-rmse:6.6493

...

[26]	validation-rmse:6.43401                                                    
[27]	validation-rmse:6.43103                                                    
[28]	validation-rmse:6.43033                                                    
[29]	validation-rmse:6.42758                                                    
[30]	validation-rmse:6.42550                                                    
[31]	validation-rmse:6.42322                                                    
[32]	validation-rmse:6.42026                                                    
[33]	validation-rmse:6.41859                                                    
[34]	validation-rmse:6.41647                                                    
[35]	validation-rmse:6.41471                                                    
[36]	validation-rmse:6.41302                                                    
[37]	validation-rmse:6.41187                                                    
[38]	validation-rmse:6.41140

[55]	validation-rmse:6.48001                                                    
[56]	validation-rmse:6.47824                                                    
[57]	validation-rmse:6.47721                                                    
[58]	validation-rmse:6.47519                                                    
[59]	validation-rmse:6.47300                                                    
[60]	validation-rmse:6.47167                                                    
[61]	validation-rmse:6.46908                                                    
[62]	validation-rmse:6.46662                                                    
[63]	validation-rmse:6.46457                                                    
[64]	validation-rmse:6.46142                                                    
[65]	validation-rmse:6.46146                                                    
[66]	validation-rmse:6.46090                                                    
[67]	validation-rmse:6.45974

[257]	validation-rmse:6.33723                                                   
[258]	validation-rmse:6.33758                                                   
[259]	validation-rmse:6.33676                                                   
[260]	validation-rmse:6.33717                                                   
[261]	validation-rmse:6.33687                                                   
[262]	validation-rmse:6.33666                                                   
[263]	validation-rmse:6.33643                                                   
[264]	validation-rmse:6.33690                                                   
[265]	validation-rmse:6.33666                                                   
[266]	validation-rmse:6.33688                                                   
[267]	validation-rmse:6.33577                                                   
[268]	validation-rmse:6.33638                                                   
[269]	validation-rmse:6.3361

[12]	validation-rmse:7.99912                                                    
[13]	validation-rmse:7.77135                                                    
[14]	validation-rmse:7.58567                                                    
[15]	validation-rmse:7.43499                                                    
[16]	validation-rmse:7.30957                                                    
[17]	validation-rmse:7.20712                                                    
[18]	validation-rmse:7.12160                                                    
[19]	validation-rmse:7.05035                                                    
[20]	validation-rmse:6.99124                                                    
[21]	validation-rmse:6.94151                                                    
[22]	validation-rmse:6.90088                                                    
[23]	validation-rmse:6.86584                                                    
[24]	validation-rmse:6.83720

[214]	validation-rmse:6.62064                                                   
[215]	validation-rmse:6.62056                                                   
[216]	validation-rmse:6.62047                                                   
[217]	validation-rmse:6.62051                                                   
[218]	validation-rmse:6.62051                                                   
[219]	validation-rmse:6.62047                                                   
[220]	validation-rmse:6.62037                                                   
[221]	validation-rmse:6.62028                                                   
[222]	validation-rmse:6.62027                                                   
[223]	validation-rmse:6.62026                                                   
[224]	validation-rmse:6.62033                                                   
[225]	validation-rmse:6.62039                                                   
[226]	validation-rmse:6.6203

In [20]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Drax")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [17]:
best_params = {
    'max_depth': 7,
    'learning_rate': 0.632127985225914,
    'reg_alpha': 0.1496160946864378,
    'reg_lambda': 0.002548368630105995,
    'min_child_weight': 1.9701119841585888,
    'objective': 'reg:linear',
    'seed': 42
}

with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(best_params)
        booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        
        with open("models/preprocessor.b", "wb") as f_out:
            pickle.dump(dv, f_out)
            
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
        
        mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[05:40:32] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:10.58935
[1]	validation-rmse:7.66643
[2]	validation-rmse:6.96423
[3]	validation-rmse:6.77737
[4]	validation-rmse:6.71612
[5]	validation-rmse:6.69616
[6]	validation-rmse:6.68535
[7]	validation-rmse:6.68073
[8]	validation-rmse:6.67271
[9]	validation-rmse:6.66765
[10]	validation-rmse:6.66223
[11]	validation-rmse:6.65816
[12]	validation-rmse:6.65080
[13]	validation-rmse:6.65118
[14]	validation-rmse:6.64755
[15]	validation-rmse:6.64487
[16]	validation-rmse:6.64253
[17]	validation-rmse:6.64205
[18]	validation-rmse:6.64072
[19]	validation-rmse:6.63950
[20]	validation-rmse:6.63458
[21]	validation-rmse:6.63195
[22]	validation-rmse:6.63029
[23]	validation-rmse:6.62500
[24]	validation-rmse:6.62039
[25]	validation-rmse:6.61683
[26]	validation-rmse:6.61360
[27]	validation-rmse:6.61060
[28]	validation-rmse:6.60773
[29]	validation-rmse:6.60427
[30]	validation-rmse

[273]	validation-rmse:6.37164
[274]	validation-rmse:6.37051
[275]	validation-rmse:6.37052
[276]	validation-rmse:6.37104
[277]	validation-rmse:6.37140
[278]	validation-rmse:6.37146
[279]	validation-rmse:6.37175
[280]	validation-rmse:6.37179
[281]	validation-rmse:6.37157
[282]	validation-rmse:6.37163
[283]	validation-rmse:6.37072
[284]	validation-rmse:6.37014
[285]	validation-rmse:6.36961
[286]	validation-rmse:6.36979
[287]	validation-rmse:6.36965
[288]	validation-rmse:6.36969
[289]	validation-rmse:6.36922
[290]	validation-rmse:6.36872
[291]	validation-rmse:6.36858
[292]	validation-rmse:6.36775
[293]	validation-rmse:6.36729
[294]	validation-rmse:6.36679
[295]	validation-rmse:6.36678
[296]	validation-rmse:6.36632
[297]	validation-rmse:6.36591
[298]	validation-rmse:6.36550
[299]	validation-rmse:6.36545
[300]	validation-rmse:6.36495
[301]	validation-rmse:6.36538
[302]	validation-rmse:6.36497
[303]	validation-rmse:6.36485
[304]	validation-rmse:6.36479
[305]	validation-rmse:6.36446
[306]	vali

[547]	validation-rmse:6.31938
[548]	validation-rmse:6.31919
[549]	validation-rmse:6.31893
[550]	validation-rmse:6.31888
[551]	validation-rmse:6.31860
[552]	validation-rmse:6.31845
[553]	validation-rmse:6.31833
[554]	validation-rmse:6.31786
[555]	validation-rmse:6.31791
[556]	validation-rmse:6.31802
[557]	validation-rmse:6.31812
[558]	validation-rmse:6.31852
[559]	validation-rmse:6.31784
[560]	validation-rmse:6.31757
[561]	validation-rmse:6.31743
[562]	validation-rmse:6.31771
[563]	validation-rmse:6.31766
[564]	validation-rmse:6.31769
[565]	validation-rmse:6.31754
[566]	validation-rmse:6.31673
[567]	validation-rmse:6.31619
[568]	validation-rmse:6.31605
[569]	validation-rmse:6.31518
[570]	validation-rmse:6.31520
[571]	validation-rmse:6.31494
[572]	validation-rmse:6.31479
[573]	validation-rmse:6.31472
[574]	validation-rmse:6.31420
[575]	validation-rmse:6.31240
[576]	validation-rmse:6.31141
[577]	validation-rmse:6.31148
[578]	validation-rmse:6.31121
[579]	validation-rmse:6.31272
[580]	vali

[821]	validation-rmse:6.30060
[822]	validation-rmse:6.30053
[823]	validation-rmse:6.30010
[824]	validation-rmse:6.29992
[825]	validation-rmse:6.30022
[826]	validation-rmse:6.30028
[827]	validation-rmse:6.30039
[828]	validation-rmse:6.30050
[829]	validation-rmse:6.30063
[830]	validation-rmse:6.30065
[831]	validation-rmse:6.30061
[832]	validation-rmse:6.30063
[833]	validation-rmse:6.29973
[834]	validation-rmse:6.29972
[835]	validation-rmse:6.29970
[836]	validation-rmse:6.29970
[837]	validation-rmse:6.30048
[838]	validation-rmse:6.30038
[839]	validation-rmse:6.30012
[840]	validation-rmse:6.29946
[841]	validation-rmse:6.30041
[842]	validation-rmse:6.30076
[843]	validation-rmse:6.30063
[844]	validation-rmse:6.30118


/home/ubuntu/anaconda3/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [18]:
import mlflow
logged_model = 'runs:/477994db86664e86b9d53d4710dd91b0/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


[06:22:39] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [19]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 477994db86664e86b9d53d4710dd91b0

In [20]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

[06:24:59] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [21]:
y_pred = xgboost_model.predict(valid)

In [22]:
y_pred[:10]

array([14.490713 ,  6.7060385, 12.827771 , 24.381386 ,  9.348514 ,
       17.515265 , 11.357995 ,  7.9936085,  9.052991 , 19.730217 ],
      dtype=float32)

In [18]:


from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        #mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        
        with open("models/preprocessor2.b", "wb") as f_out:
            pickle.dump(dv, f_out)
            
        mlflow.log_artifact("models/preprocessor2.b", artifact_path="preprocessor")
        
        mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
        



/home/ubuntu/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
